In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datasist.structdata import detect_outliers

%matplotlib inline
sns.set(rc={'figure.figsize': [10, 10]}, font_scale=1.3)
pd.set_option('mode.chained_assignment', None)

In [2]:
df = pd.read_csv("Sample - Superstore.csv", encoding='windows-1252')
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/08/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/08/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,06/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


### Preprocessing and feature engineering

In [3]:
# function for Removal of Outliers
def remove_outlier(dataset,k=3.33):
    for col in dataset.columns:
            if (dataset[col].dtype=="int64" or dataset[col].dtype=="float64"):
                mean = dataset[col].mean()
                global ds      
                std = dataset[col].std()    
                outlier = [i for i in dataset[col] if (i > mean - k * std)]
                outlier = [i for i in outlier if (i < mean + k * std)]       
                ds = dataset.loc[dataset[col].isin(outlier)]
remove_outlier(df,k=3.33)

In [4]:
# Encode dates
def encode_dates(df, column):
    df = df.copy()
    df[column] = pd.to_datetime(df[column])
    df[column + '_year'] = df[column].apply(lambda x: x.year)
    df[column + '_month'] = df[column].apply(lambda x: x.month)
    df[column + '_day'] = df[column].apply(lambda x: x.day)
    df = df.drop(column, axis=1)
    return df

# Work with categorical data
def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

# Final function for feature engineering
def preprocess_inputs(df):
    df = df.copy()
    
    # Drop unnecessary columns
    df = df.drop(['Row ID', 'Customer Name', 'Country', 'Product Name', 'Postal Code',  'City', 'Product ID'], axis=1)
    
    # Drop customer-specific feature columns
    df = df.drop(['Order ID', 'Customer ID'], axis=1)
    
    # Extract date features
    df = encode_dates(df, column='Order Date')
    df = encode_dates(df, column='Ship Date')
    
    # One-hot encode categorical features
    for column in ['Ship Mode', 'Segment','State', 'Region', 'Category', 'Sub-Category']:
        df = onehot_encode(df, column=column)
    
    return df


In [5]:
df_preprocessing = preprocess_inputs(df)
df_preprocessing

,Sales,Quantity,Discount,Profit,Order Date_year,Order Date_month,Order Date_day,Ship Date_year,Ship Date_month,Ship Date_day,...,Sub-Category_Envelopes,Sub-Category_Fasteners,Sub-Category_Furnishings,Sub-Category_Labels,Sub-Category_Machines,Sub-Category_Paper,Sub-Category_Phones,Sub-Category_Storage,Sub-Category_Supplies,Sub-Category_Tables
0,261.9600,2,0.00,41.9136,2016,11,8,2016,11,11,...,0,0,0,0,0,0,0,0,0,0
1,731.9400,3,0.00,219.5820,2016,11,8,2016,11,11,...,0,0,0,0,0,0,0,0,0,0
2,14.6200,2,0.00,6.8714,2016,6,12,2016,6,16,...,0,0,0,1,0,0,0,0,0,0
3,957.5775,5,0.45,-383.0310,2015,10,11,2015,10,18,...,0,0,0,0,0,0,0,0,0,1
4,22.3680,2,0.20,2.5164,2015,10,11,2015,10,18,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,25.2480,3,0.20,4.1028,2014,1,21,2014,1,23,...,0,0,1,0,0,0,0,0,0,0
9990,91.9600,2,0.00,15.6332,2017,2,26,2017,3,3,...,0,0,1,0,0,0,0,0,0,0
9991,258.5760,2,0.20,19.3932,2017,2,26,2017,3,3,...,0,0,0,0,0,0,1,0,0,0
9992,29.6000,4,0.00,13.3200,2017,2,26,2017,3,3,...,0,0,0,0,0,1,0,0,0,0


### Train-test split

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
# Split df into X and y
y = df_preprocessing['Sales']
x = df_preprocessing.drop('Sales', axis=1)

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15)

### Scalling

In [9]:
from sklearn.preprocessing import StandardScaler

scaler= StandardScaler()

scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

### Modeling

In [10]:
from xgboost import XGBRegressor

model = XGBRegressor()

model.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [11]:
model.score(x_train, y_train)

0.9902523808470798

In [12]:
y_preds = model.predict(x_test)
y_preds

array([ 10.816155, 125.09793 , 107.270805, ..., 173.55763 ,  70.73097 ,
        35.108166], dtype=float32)

In [13]:
y_test

1210      8.544
8218     31.776
3800    102.592
9079      5.800
720     146.820
         ...   
3571      7.992
9607     21.600
8011    125.944
2235    190.860
4350     16.768
Name: Sales, Length: 1500, dtype: float64

In [14]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [15]:
np.sqrt(mean_squared_error(y_test, y_preds))

233.17118921756838

In [16]:
r2_score(y_test, y_preds)

0.8489770895785074

In [17]:
## score 
print('Train Score', r2_score(y_train, model.predict(x_train)))
print('Validation Score', r2_score(y_test, y_preds))

Train Score 0.9902523808470798
Validation Score 0.8489770895785074


In [18]:
mean_absolute_error(y_test, y_preds)

73.72653470949449

### Saving model

In [19]:
import joblib

In [20]:
joblib.dump(model, 'model.h5')

['model.h5']

In [21]:
joblib.dump(scaler, 'scaler.h5')

['scaler.h5']

In [22]:
x.columns

Index(['Quantity', 'Discount', 'Profit', 'Order Date_year', 'Order Date_month',
       'Order Date_day', 'Ship Date_year', 'Ship Date_month', 'Ship Date_day',
       'Ship Mode_First Class', 'Ship Mode_Same Day', 'Ship Mode_Second Class',
       'Ship Mode_Standard Class', 'Segment_Consumer', 'Segment_Corporate',
       'Segment_Home Office', 'State_Alabama', 'State_Arizona',
       'State_Arkansas', 'State_California', 'State_Colorado',
       'State_Connecticut', 'State_Delaware', 'State_District of Columbia',
       'State_Florida', 'State_Georgia', 'State_Idaho', 'State_Illinois',
       'State_Indiana', 'State_Iowa', 'State_Kansas', 'State_Kentucky',
       'State_Louisiana', 'State_Maine', 'State_Maryland',
       'State_Massachusetts', 'State_Michigan', 'State_Minnesota',
       'State_Mississippi', 'State_Missouri', 'State_Montana',
       'State_Nebraska', 'State_Nevada', 'State_New Hampshire',
       'State_New Jersey', 'State_New Mexico', 'State_New York',
       'State_Nort